In [1]:
!ls ./input/* -lh

-rw-r--r-- 1 aistudio aistudio  91M 3月  29 18:02 ./input/corpus.tsv
-rw-r--r-- 1 aistudio aistudio  24K 3月  29 18:00 ./input/dev.query.txt
-rw-r--r-- 1 aistudio aistudio 1.3M 3月  29 18:00 ./input/qrels.train.tsv
-rw-r--r-- 1 aistudio aistudio 2.3M 3月  29 18:00 ./input/train.query.txt


In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook

In [3]:
corpus_data = pd.read_csv( "./input/corpus.tsv", sep="\t", names=["doc", "title"])
dev_data = pd.read_csv("./input/dev.query.txt", sep="\t", names=["query", "title"])
train_data = pd.read_csv("./input/train.query.txt", sep="\t", names=["query", "title"])
qrels = pd.read_csv("./input/qrels.train.tsv", sep="\t", names=["query", "doc"])

In [4]:
corpus_data = corpus_data.set_index("doc")
dev_data = dev_data.set_index("query")
train_data = train_data.set_index("query")
qrels = qrels.set_index("query")

In [5]:
qrels.head()

,doc
query,
1,679139
2,35343
3,781652
4,557516
5,588014


In [6]:
for idx in range(1, 20):
    print(
        train_data.loc[idx]["title"],
        "\t",
        corpus_data.loc[qrels.loc[idx].ravel()[0]]["title"],
    )

美赞臣亲舒一段 	 领券满减】美赞臣安婴儿A+亲舒 婴儿奶粉1段850克 0-12个月宝宝
慱朗手动料理机 	 Braun/博朗 MQ3035/3000/5025料理棒手持小型婴儿辅食家用搅拌机
電力貓 	 小米WiFi电力猫无线路由器套装一对300M穿墙宝家用信号增强扩展器
掏夹缝工具 	 电梯地坎清洁工具除灰尘神器轿厢门槽缝隙掏勺维保打扫奥的斯三菱
飞推vip 	 飞逗推拍 店主邀请码 去水印 创意视频一键制作视频
多功能托地把 	 免手洗拖把家用一拖净刮刮乐干湿两用懒人拖平板墩布托帕拖地神器
充气浮力袖 	 学游泳神器装备充气腰背漂水袖浮臂三角浮力儿童游泳辅助工具大人
盒马花胶鸡汤锅 	 盒马鲜生工坊代购 花胶奶冻150g 入口Q弹 奶味浓郁 香甜丝滑
塞塞乐 	 婴儿童玩具6个月以上8宝宝益智早教0一1岁男孩女孩六9月十7新生礼
广汽传祺gs5挡风遮雨条子 	 2021款广汽传祺GS5晴雨挡遮雨板传奇GS5配件车窗雨眉防雨条挡雨板
冰墩敦人偶服装 	 灯笼布偶熊猫道具服装吉祥物人穿玩偶宣传传单服活动人偶服装
寵物罐頭密封蓋 	 仁可宠物 猫罐头保鲜盒密封盖防潮可加热猫咪罐头勺喂食勺猫用品
15 蒸汽爱美克闸阀 	 埃美柯 8135闸阀304不锈钢蒸汽用闸阀中型Z15W-16P耐温腐蚀4分6分
电动切面机 	 复兴牌面条机电动家用不锈钢压面机多功能半全自动四种面条DMT-6
医用震动排痰机 	 普门排痰机振动背心式慢阻肺支气管扩张肺气肿医用咳痰祛痰神器
草莓盆专用夹 	 大棚草莓钩盆器新款农具草莓采摘神器摘取自如温室水果铁丝钩子
lg洗烘套装 	 LG RC90V9AV2W RC90V9JV2W RH10进口9/10KG热泵双变频干衣烘干机
芝士脆 	 山居小食 芝士小脆棒 香酥小零食 罐装 110g包邮
笔记本应用书籍 	 ThinkPad笔记本电脑应用技术精粹


In [7]:
import jieba
" ".join(jieba.cut("美赞臣亲舒一段"))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.796 seconds.
Prefix dict has been built successfully.


'美赞臣 亲舒 一段'

In [8]:
def title_cut(x):
    return list(jieba.cut(x))

from joblib import Parallel, delayed

corpus_title = Parallel(n_jobs=4)(delayed(title_cut)(title) for title in corpus_data["title"])
train_title = Parallel(n_jobs=4)(delayed(title_cut)(title) for title in train_data["title"])
dev_title = Parallel(n_jobs=4)(delayed(title_cut)(title) for title in dev_data["title"])

r_corpus_title = Parallel(n_jobs=4)(delayed(title_cut)(title[::-1]) for title in corpus_data["title"])
r_train_title = Parallel(n_jobs=4)(delayed(title_cut)(title[::-1]) for title in train_data["title"])
r_dev_title = Parallel(n_jobs=4)(delayed(title_cut)(title[::-1]) for title in dev_data["title"])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built successfully.
Loading model cost 0.726 seconds.
Prefix dict has been built successfully.
Loading model cost 0.749 seconds.
Prefix dict has been built successfully.
Loading model cost 0.742 seconds.
Prefix dict has been built successfully.


In [9]:
! pip install gensim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 2.1 MB/s eta 0:00:00


In [10]:
corpus_title[0], r_corpus_title[0], 

(['铂',
  '盛弹',
  '盖',
  '文艺',
  '保温杯',
  '学生',
  '男女',
  '情侣',
  '车载',
  '时尚',
  '英文',
  '锁扣',
  '不锈钢',
  '真空',
  '水杯'],
 ['杯水',
  '空真',
  '钢锈',
  '不',
  '扣锁',
  '文英尚',
  '时载',
  '车侣',
  '情女',
  '男生',
  '学杯',
  '温保',
  '艺文',
  '盖弹',
  '盛',
  '铂'])

In [11]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

_L = []
for i in [
    corpus_title, r_corpus_title,
    train_title, r_train_title,
    dev_title, r_dev_title,
]:
    _L += list(i)

if 1:
    model = Word2Vec(
        sentences=_L,
        vector_size=128,
        window=5,
        min_count=1,
        workers=4,
    )
    # model.save("word2vec.model")

In [12]:
model.wv.most_similar("小天鹅")

[('海尔', 0.9109137058258057),
 ('惠而浦', 0.8970657587051392),
 ('波轮', 0.8927889466285706),
 ('荣事达', 0.8710645437240601),
 ('三洋', 0.8654263615608215),
 ('志高', 0.8652589917182922),
 ('金羚', 0.8613708019256592),
 ('伊莱克斯', 0.8563185334205627),
 ('容声', 0.8506056666374207),
 ('新飞', 0.849298357963562)]

In [13]:
model.wv.index_to_key[:10]

[' ', '/', '新款', '女', '-', '新', '小', '1202', '2021', '年']

In [14]:
model.wv.key_to_index["女"]

3

In [15]:
train_w2v_ids = [[model.wv.key_to_index[xx] for xx in x] for x in train_title]
corpus_w2v_ids = [[model.wv.key_to_index[xx] for xx in x] for x in corpus_title]
dev_w2v_ids = [[model.wv.key_to_index[xx] for xx in x] for x in dev_title]

# all_text = " ".join(train_data["title"])
# all_query_word = list(jieba.cut(all_text))
# all_query_word = [x for x in all_query_word if len(x) >= 2]
# all_query_ids = [model.wv.key_to_index[xx] for xx in all_query_word]

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

idf = TfidfVectorizer(analyzer=lambda x: x)
idf.fit(_L)

TfidfVectorizer(analyzer=<function <lambda> at 0x7f9084094560>)

In [17]:
idf.idf_, len(idf.idf_)

(array([ 2.46365496,  8.57892401,  7.69932303, ..., 14.50762623,
        14.50762623, 14.50762623]),
 2151151)

In [18]:
token = np.array(idf.get_feature_names())
drop_token = token[np.where(idf.idf_ < 10)[0]]
drop_token = list(set(drop_token))
drop_token += ['领券']

In [19]:
drop_token_ids = [model.wv.key_to_index[x] for x in drop_token]

In [20]:
[idf.idf_[idf.vocabulary_[xx]] for xx in train_title[0]]

[11.777597123888626, 14.21994415925783, 11.236790667910698]

In [21]:
def unsuper_w2c_encoding(s, pooling="max"):
    feat = []
    corpus_query_word = [x for x in s if x not in drop_token_ids]
    if len(corpus_query_word) == 0:
        return np.zeros(128)
    
    feat = model.wv[corpus_query_word]

    if pooling == "max":
        return np.array(feat).max(0)
    if pooling == "avg":
        return np.array(feat).mean(0)


# def unsuper_w2c_encoding(s, pooling="avg", debug=False):
#     feat = []
    
#     # corpus_query_word = list(set(s) & set(all_query_ids))

#     for w in s:
        
#         if idf.idf_[idf.vocabulary_[w]] > 11:
#             if debug:
#                 print(w)
#             feat.append(model.wv[w])
        
#     if len(feat) == 0:
#         return np.zeros(128)


#     if pooling == "max":
#         return np.array(feat).max(0)
#     if pooling == "avg":
#         return np.array(feat).mean(0)

In [22]:
unsuper_w2c_encoding(train_w2v_ids[0])

array([ 0.01410294, -0.03388168,  0.17070074, -0.01582576,  0.5727828 ,
        0.02748853,  0.1402859 , -0.03211262,  0.26610637, -0.01375053,
        0.40250638,  0.3324122 , -0.00461612,  0.01067858,  0.53587514,
        0.05884365, -0.04697658,  0.9005212 , -0.04430126,  0.6041017 ,
        0.3899894 ,  0.23006596,  0.1375492 ,  0.12018747,  0.00566164,
        0.00358749,  0.00785051,  0.16608948,  0.29385695, -0.01588646,
       -0.02892047, -0.01299607,  0.1998594 ,  0.2696155 , -0.03007177,
        0.07294509,  0.38134786, -0.0174321 ,  0.27576542,  0.17722964,
        0.00570896,  0.51388127,  0.01622434,  0.03703029,  0.51875967,
        0.20672448,  0.0047464 , -0.00643368,  0.0571991 , -0.02023711,
        0.12713301,  0.50466305,  0.32924235,  0.00173132,  0.03059099,
        0.6576533 ,  0.05095484, -0.01399578,  0.02472398,  0.32039407,
       -0.03275841, -0.04458891,  0.10853255,  0.29226935,  0.5685516 ,
        0.0047483 ,  0.5078857 ,  0.18432808,  0.01453441, -0.03

In [23]:
_ = unsuper_w2c_encoding(corpus_w2v_ids[679139-1])

In [24]:
from tqdm import tqdm_notebook
# [corpus_w2v_ids[x] for x in qrels['doc'].values[:100] - 1]

corpus_mean_feat = [unsuper_w2c_encoding(s) for s in tqdm_notebook(corpus_w2v_ids[:])]
corpus_mean_feat = np.vstack(corpus_mean_feat)

train_mean_feat = [unsuper_w2c_encoding(s) for s in tqdm_notebook(train_w2v_ids[:])]
train_mean_feat = np.vstack(train_mean_feat)

dev_mean_feat = [unsuper_w2c_encoding(s) for s in tqdm_notebook(dev_w2v_ids[:])]
dev_mean_feat = np.vstack(dev_mean_feat)

In [25]:
from sklearn.preprocessing import normalize

corpus_mean_feat = normalize(corpus_mean_feat)
train_mean_feat = normalize(train_mean_feat)
dev_mean_feat = normalize(dev_mean_feat)

In [26]:
mrr = []
for idx in tqdm_notebook(range(1, 100)):
    dis = np.dot(train_mean_feat[idx - 1], corpus_mean_feat.T)
    ids = np.argsort(dis)[::-1]
    
    # print(train_title[idx-1], corpus_data.loc[qrels.loc[idx].ravel()[0]]["title"],  dis[qrels.loc[idx].ravel()-1])
    # print(corpus_title[ids[0]])
    # mrr.append(1/(np.where(ids == qrels.loc[idx].ravel()[0] - 1)[0][0] + 1))
    # break
    # print('')
    # mrr.append(ids[0]==idx+999)

In [27]:
np.mean(mrr)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [28]:
with open('query_embedding', 'w') as up :
    for id, feat in zip(dev_data.index, dev_mean_feat):
        up.write('{0}\t{1}\n'.format(id, ','.join([str(x)[:6] for x in feat])))
        
with open('doc_embedding', 'w') as up :
    for id, feat in zip(corpus_data.index, corpus_mean_feat):
        up.write('{0}\t{1}\n'.format(id, ','.join([str(x)[:6] for x in feat])))

In [29]:
! python data_check.py

********** Checking query_embedding ...
Check done!

********** Checking doc_embedding ...
Check done!



In [30]:
! tar zcvf foo.tar.gz doc_embedding query_embedding

doc_embedding
query_embedding
